In [55]:
import zipfile
import glob
import zarr
from netCDF4 import Dataset
from pathlib import Path
import datetime

# Convert a netcdf file to zarr

** install pyutils to get the data_read.download function and tracking to get the download and single_zarr functions**

In [2]:
!conda install -y fastparquet python-snappy

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/phil/mini36:
#
fastparquet               0.1.0                    py36_2    conda-forge
python-snappy             0.5.1                    py36_0    conda-forge


### here are two modules I've uploaded to my anaconda channel

In [3]:
!conda install -y pyutils tracking -c phaustin

Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/phil/mini36:

The following NEW packages will be INSTALLED:

    tracking: 0.1-py_0 phaustin

tracking-0.1-p 100% |################################| Time: 0:00:00   9.22 MB/s


** download the netcdf files and unzip **

Use my [data_read](https://github.com/phaustin/pythonlibs/blob/master/pyutils/pyutils/data_read.py) utility to download the netcdf files in a zipfile

In [4]:
from pyutils.data_read import download

In [5]:
the_zip = 'test.zip'
download(the_zip)

trying https://clouds.eos.ubc.ca/~phil/courses/atsc301/downloads/test.zip
writing to: test.zip

test.zip already exists
and is 95492218 bytes
will not overwrite



### now unzip to get the wrf netcdf files

In [6]:
with zipfile.ZipFile(the_zip,"r") as zip_ref:
    zip_ref.extractall(".")

### now convert from netcdf to zarr with the module  [tracking.single_zarr](https://github.com/phaustin/tracking/blob/master/tracking/single_zarr.py)

In [7]:
from tracking.single_zarr import convert_file, dump_meta

In [8]:
nc_files = glob.glob('*.nc')  
for the_file in nc_files:
    nc_path=Path(the_file)
    #
    # make the output directory the same as the input nc name
    # with .zarr instead of .nc
    #
    zarr_dir=nc_path.with_suffix('.zarr')
    convert_file(the_file,zarr_dir)
    #
    # store the metadata from the netcdf file in a nested dictionary
    #
    meta_dict=dump_meta(the_file)
    store = zarr.DirectoryStore(zarr_dir)
    #
    # open the file in append mode and transfer the metadata
    #
    the_group=zarr.hierarchy.open_group(store=store, mode='a',
                                        synchronizer=None, path=None)
    for key,value in meta_dict['global'].items():
        the_group.attrs[key]=value
    del meta_dict['global']
    for var,var_dict in meta_dict.items():
        for key, value in var_dict.items():
            the_group[var].attrs[key]=value
            print(var,list(the_group[var].attrs.keys()))


variable: Times
variable: XLAT
variable: XLONG
variable: LU_INDEX
variable: ZNU
variable: ZNW
variable: ZS
variable: DZS
variable: U
variable: V
variable: W
variable: PH
variable: PHB
variable: T
variable: HFX_FORCE
variable: LH_FORCE
variable: TSK_FORCE
variable: HFX_FORCE_TEND
variable: LH_FORCE_TEND
variable: TSK_FORCE_TEND
variable: MU
variable: MUB
variable: P
variable: PB
variable: P_HYD
variable: Q2
variable: T2
variable: TH2
variable: PSFC
variable: U10
variable: V10
variable: ZETATOP
variable: ITIMESTEP
variable: XTIME
variable: QVAPOR
variable: QCLOUD
variable: QRAIN
variable: QICE
variable: QSNOW
variable: TSLB
variable: SMOIS
variable: SH2O
variable: SMCREL
variable: SFROFF
variable: IVGTYP
variable: ISLTYP
variable: VEGFRA
variable: GRDFLX
variable: ACGRDFLX
variable: ACSNOM
variable: SNOW
variable: SNOWH
variable: SSTSK
variable: COSZEN
variable: LAI
variable: Z0
variable: VAR
variable: MAPFAC_M
variable: MAPFAC_U
variable: MAPFAC_V
variable: MAPFAC_MX
variable: MAPFAC_MY

P ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
P ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
PB ['FieldType']
PB ['FieldType', 'MemoryOrder']
PB ['FieldType', 'MemoryOrder', 'description']
PB ['FieldType', 'MemoryOrder', 'description', 'units']
PB ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
PB ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
P_HYD ['FieldType']
P_HYD ['FieldType', 'MemoryOrder']
P_HYD ['FieldType', 'MemoryOrder', 'description']
P_HYD ['FieldType', 'MemoryOrder', 'description', 'units']
P_HYD ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
P_HYD ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
Q2 ['FieldType']
Q2 ['FieldType', 'MemoryOrder']
Q2 ['FieldType', 'MemoryOrder', 'description']
Q2 ['FieldType', 'MemoryOrder', 'description', 'units']
Q2 ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
Q2 ['FieldType

ACSNOM ['FieldType', 'MemoryOrder', 'description']
ACSNOM ['FieldType', 'MemoryOrder', 'description', 'units']
ACSNOM ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
ACSNOM ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
SNOW ['FieldType']
SNOW ['FieldType', 'MemoryOrder']
SNOW ['FieldType', 'MemoryOrder', 'description']
SNOW ['FieldType', 'MemoryOrder', 'description', 'units']
SNOW ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
SNOW ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
SNOWH ['FieldType']
SNOWH ['FieldType', 'MemoryOrder']
SNOWH ['FieldType', 'MemoryOrder', 'description']
SNOWH ['FieldType', 'MemoryOrder', 'description', 'units']
SNOWH ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
SNOWH ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
SSTSK ['FieldType']
SSTSK ['FieldType', 'MemoryOrder']
SSTSK ['FieldType', 'MemoryOrder', 'descripti

SWNORM ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
OLR ['FieldType']
OLR ['FieldType', 'MemoryOrder']
OLR ['FieldType', 'MemoryOrder', 'description']
OLR ['FieldType', 'MemoryOrder', 'description', 'units']
OLR ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
OLR ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
XLAT_U ['FieldType']
XLAT_U ['FieldType', 'MemoryOrder']
XLAT_U ['FieldType', 'MemoryOrder', 'description']
XLAT_U ['FieldType', 'MemoryOrder', 'description', 'units']
XLAT_U ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
XLAT_U ['FieldType', 'MemoryOrder', 'coordinates', 'description', 'stagger', 'units']
XLONG_U ['FieldType']
XLONG_U ['FieldType', 'MemoryOrder']
XLONG_U ['FieldType', 'MemoryOrder', 'description']
XLONG_U ['FieldType', 'MemoryOrder', 'description', 'units']
XLONG_U ['FieldType', 'MemoryOrder', 'description', 'stagger', 'units']
XLONG_U ['FieldType', 'MemoryOrder', '

variable: DateTime
variable: year
variable: month
variable: day
variable: hour
variable: minute
variable: pressure
variable: eta
variable: p_top
variable: lat
variable: lon
variable: Z_sfc
variable: SST
variable: T_sfc
variable: p_sfc
variable: slp
variable: T_2m
variable: Td_2m
variable: r_v_2m
variable: q_2m
variable: rh_2m
variable: u_10m_gr
variable: v_10m_gr
variable: u_10m_tr
variable: v_10m_tr
variable: ws_10m
variable: wd_10m
variable: precip_g
variable: precip_c
variable: pblh
variable: r_cloud
variable: r_rain
variable: r_ice
variable: r_snow
variable: Z_p
variable: T_p
variable: Td_p
variable: r_v_p
variable: q_p
variable: u_tr_p
variable: v_tr_p
variable: w_p
variable: albedo
variable: SH
variable: LH
variable: SH_acc
variable: LH_acc
variable: MH
variable: u_star
variable: LWP
variable: IWP
variable: LandMask
variable: LandUse
copying global attribute:  title
copying global attribute:  Conventions
copying global attribute:  source
copying global attribute:  notes
copying g

Td_p ['long_name']
Td_p ['long_name', 'standard_name']
Td_p ['long_name', 'standard_name', 'units']
Td_p ['_FillValue', 'long_name', 'standard_name', 'units']
Td_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
r_v_p ['long_name']
r_v_p ['long_name', 'standard_name']
r_v_p ['long_name', 'standard_name', 'units']
r_v_p ['_FillValue', 'long_name', 'standard_name', 'units']
r_v_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
q_p ['long_name']
q_p ['long_name', 'standard_name']
q_p ['long_name', 'standard_name', 'units']
q_p ['_FillValue', 'long_name', 'standard_name', 'units']
q_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
u_tr_p ['long_name']
u_tr_p ['long_name', 'standard_name']
u_tr_p ['long_name', 'standard_name', 'units']
u_tr_p ['_FillValue', 'long_name', 'standard_name', 'units']
u_tr_p ['_FillValue', 'coordinates', 'long_name', 'standard_name', 'units']
v_tr_p ['long_name']
v_tr_p ['long_name', 'standard_name']

In [9]:
!ls -ltd *zarr

drwxr-xr-x   58 phil  staff  1972 Jul  4 13:13 wrfout_d03_2016-01-04_09:00:00_post.zarr
drwxr-xr-x  121 phil  staff  4114 Jul  4 13:13 wrfout_d03_2016-01-04_09:00:00.zarr


In [10]:
!du -h wrfout*

148M	wrfout_d03_2016-01-04_09:00:00.nc
108K	wrfout_d03_2016-01-04_09:00:00.zarr/ACGRDFLX
152K	wrfout_d03_2016-01-04_09:00:00.zarr/ACHFX
152K	wrfout_d03_2016-01-04_09:00:00.zarr/ACLHF
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/ACSNOM
 72K	wrfout_d03_2016-01-04_09:00:00.zarr/ALBEDO
 88K	wrfout_d03_2016-01-04_09:00:00.zarr/CLAT
2.2M	wrfout_d03_2016-01-04_09:00:00.zarr/CLDFRA
 96K	wrfout_d03_2016-01-04_09:00:00.zarr/COSALPHA
 88K	wrfout_d03_2016-01-04_09:00:00.zarr/COSZEN
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/DZS
 64K	wrfout_d03_2016-01-04_09:00:00.zarr/EMISS
132K	wrfout_d03_2016-01-04_09:00:00.zarr/GLW
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/GRAUPELNC
108K	wrfout_d03_2016-01-04_09:00:00.zarr/GRDFLX
 32K	wrfout_d03_2016-01-04_09:00:00.zarr/HAILNC
152K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX_FORCE
 12K	wrfout_d03_2016-01-04_09:00:00.zarr/HFX_FORCE_TEND
108K	wrfout_d03_2016-01-04_09:00:00.zarr/HGT
 40K	wrfout_d03_2016-01-04_09:00:00.zarr/ISLTYP


# Convert a csv file to a pandas dataframe

In [11]:
the_csv = '684text.csv'
download(the_csv)

trying https://clouds.eos.ubc.ca/~phil/courses/atsc301/downloads/684text.csv
writing to: 684text.csv

684text.csv already exists
and is 31785 bytes
will not overwrite



## read the csv file into a dataframe named out_df, and write to a .parq file

In [12]:
from fastparquet import write, ParquetFile
import pandas as pd
in_file=Path(the_csv)
out_file=in_file.with_suffix('.parq')
print(f'here is the output name {str(out_file)}')
out_df=pd.read_csv(str(in_file))
write(str(out_file),out_df,object_encoding='utf8',compression='SNAPPY')

here is the output name 684text.parq


In [53]:
out_df.head()

,fcst_date,fcst_time,t2m,th2m,q2,u10,v10,rainacc,snowacc,slp,...,u700,u500,v925,v850,v700,v500,w925,w850,w700,w500
0,2016-01-01,09:00:00,-8.92065,278.548,-0.001062,2.29513,3.09734,0,0,1027.43,...,6.33520,4.79451,9.969210e+36,9.969210e+36,10.97430,11.9743,9.969210e+36,9.969210e+36,-0.012858,0.155495
1,2016-01-01,10:00:00,-7.63623,279.864,-0.000810,2.41331,3.63232,0,0,1027.98,...,6.79529,4.85847,9.969210e+36,9.969210e+36,11.00380,11.5140,9.969210e+36,9.969210e+36,-0.041467,0.154532
2,2016-01-01,11:00:00,-7.21616,280.320,-0.000737,2.68491,3.57354,0,0,1028.01,...,6.42482,4.41733,9.969210e+36,9.969210e+36,10.23570,11.2785,9.969210e+36,9.969210e+36,-0.005499,0.126797
3,2016-01-01,12:00:00,-6.85010,280.740,-0.000613,2.92674,3.47312,0,0,1027.13,...,5.71057,3.92283,9.969210e+36,9.969210e+36,9.58452,11.6122,9.969210e+36,9.969210e+36,0.013336,0.065372
4,2016-01-01,13:00:00,-6.09811,281.544,-0.000488,2.97867,3.68480,0,0,1026.92,...,4.44597,3.41606,9.969210e+36,9.969210e+36,9.47761,11.9958,9.969210e+36,9.969210e+36,0.039681,-0.016996


### Read it back in to make sure it's right

In [15]:
round_trip_df=ParquetFile(str(out_file)).to_pandas()
print('here is the round trip')
round_trip_df

here is the round trip


,fcst_date,fcst_time,t2m,th2m,q2,u10,v10,rainacc,snowacc,slp,...,u700,u500,v925,v850,v700,v500,w925,w850,w700,w500
0,2016-01-01,09:00:00,-8.920650,278.548,-0.001062,2.295130,3.097340,0,0,1027.43,...,6.335200,4.794510,9.969210e+36,9.969210e+36,10.974300,11.974300,9.969210e+36,9.969210e+36,-0.012858,0.155495
1,2016-01-01,10:00:00,-7.636230,279.864,-0.000810,2.413310,3.632320,0,0,1027.98,...,6.795290,4.858470,9.969210e+36,9.969210e+36,11.003800,11.514000,9.969210e+36,9.969210e+36,-0.041467,0.154532
2,2016-01-01,11:00:00,-7.216160,280.320,-0.000737,2.684910,3.573540,0,0,1028.01,...,6.424820,4.417330,9.969210e+36,9.969210e+36,10.235700,11.278500,9.969210e+36,9.969210e+36,-0.005499,0.126797
3,2016-01-01,12:00:00,-6.850100,280.740,-0.000613,2.926740,3.473120,0,0,1027.13,...,5.710570,3.922830,9.969210e+36,9.969210e+36,9.584520,11.612200,9.969210e+36,9.969210e+36,0.013336,0.065372
4,2016-01-01,13:00:00,-6.098110,281.544,-0.000488,2.978670,3.684800,0,0,1026.92,...,4.445970,3.416060,9.969210e+36,9.969210e+36,9.477610,11.995800,9.969210e+36,9.969210e+36,0.039681,-0.016996
5,2016-01-01,14:00:00,-5.970730,281.723,-0.000520,2.856720,3.778740,0,0,1026.53,...,3.883530,2.952190,9.969210e+36,9.969210e+36,9.421140,11.828100,9.969210e+36,9.969210e+36,0.065154,-0.086605
6,2016-01-01,15:00:00,-5.387820,282.320,-0.000376,2.913880,4.069470,0,0,1027.08,...,3.435230,2.585820,9.969210e+36,9.969210e+36,9.517270,11.336800,9.969210e+36,9.969210e+36,0.068575,-0.102264
7,2016-01-01,16:00:00,-5.320280,282.396,-0.000389,2.817700,4.102980,0,0,1027.12,...,3.289400,2.400470,9.969210e+36,9.969210e+36,9.484370,10.668300,9.969210e+36,9.969210e+36,0.058966,-0.111161
8,2016-01-01,17:00:00,-4.862400,282.921,-0.000253,2.668770,4.065260,0,0,1026.75,...,3.219480,2.151470,9.969210e+36,9.969210e+36,9.707330,10.273600,9.969210e+36,9.969210e+36,0.052912,-0.115268
9,2016-01-01,18:00:00,0.015717,288.070,0.000554,2.181260,4.025910,0,0,1026.49,...,2.260150,1.188190,9.969210e+36,9.969210e+36,9.860000,10.422000,9.969210e+36,9.969210e+36,0.014137,-0.064144


## Convert the date and time strings into a Unix timestamp, assuming we're in the Pacific timezone

In [20]:
!conda install -y python-dateutil

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/phil/mini36:
#
python-dateutil           2.6.0                    py36_0    conda-forge


### the dateutil module has a string parser that turns text into datetime objects

In [21]:
import dateutil as dt

### convert the text strings date-time to a datetime object

In [26]:
date_time=zip(round_trip_df['fcst_date'],round_trip_df['fcst_time'])
the_date=[f'{the_date}-{the_time}' for the_date,the_time in date_time]

In [42]:
datetimes=[dt.parser.parse(item) for item in the_date]

### assign all dates to US/Pacific timezone

In [36]:
import pytz
from pytz import timezone

In [ ]:
pac=timezone('US/Pacific')
datetimes_pac=[pac.localize(item) for item in datetimes]

### Convert from pacific to utc

In [43]:
datetimes_utc=[item.astimezone(pytz.utc) for item in datetimes_pac]

### Convert from utc to unix timestamp

In [46]:
timestamps = [int(item.strftime('%s')) for item in datetimes_utc]
timestamps[:4]

[1451696400, 1451700000, 1451703600, 1451707200]

In [47]:
round_trip_df['timestamp']=pd.Series(timestamps)

In [52]:
round_trip_df.head()

,fcst_date,fcst_time,t2m,th2m,q2,u10,v10,rainacc,snowacc,slp,...,u500,v925,v850,v700,v500,w925,w850,w700,w500,timestamp
0,2016-01-01,09:00:00,-8.92065,278.548,-0.001062,2.29513,3.09734,0,0,1027.43,...,4.79451,9.969210e+36,9.969210e+36,10.97430,11.9743,9.969210e+36,9.969210e+36,-0.012858,0.155495,1451696400
1,2016-01-01,10:00:00,-7.63623,279.864,-0.000810,2.41331,3.63232,0,0,1027.98,...,4.85847,9.969210e+36,9.969210e+36,11.00380,11.5140,9.969210e+36,9.969210e+36,-0.041467,0.154532,1451700000
2,2016-01-01,11:00:00,-7.21616,280.320,-0.000737,2.68491,3.57354,0,0,1028.01,...,4.41733,9.969210e+36,9.969210e+36,10.23570,11.2785,9.969210e+36,9.969210e+36,-0.005499,0.126797,1451703600
3,2016-01-01,12:00:00,-6.85010,280.740,-0.000613,2.92674,3.47312,0,0,1027.13,...,3.92283,9.969210e+36,9.969210e+36,9.58452,11.6122,9.969210e+36,9.969210e+36,0.013336,0.065372,1451707200
4,2016-01-01,13:00:00,-6.09811,281.544,-0.000488,2.97867,3.68480,0,0,1026.92,...,3.41606,9.969210e+36,9.969210e+36,9.47761,11.9958,9.969210e+36,9.969210e+36,0.039681,-0.016996,1451710800


### overwrite set with new timestamp column

In [49]:
write(str(out_file),round_trip_df,object_encoding='utf8',compression='SNAPPY')

### read back in

In [50]:
round_trip_df=ParquetFile(str(out_file)).to_pandas()

In [51]:
round_trip_df.head()

,fcst_date,fcst_time,t2m,th2m,q2,u10,v10,rainacc,snowacc,slp,...,u500,v925,v850,v700,v500,w925,w850,w700,w500,timestamp
0,2016-01-01,09:00:00,-8.92065,278.548,-0.001062,2.29513,3.09734,0,0,1027.43,...,4.79451,9.969210e+36,9.969210e+36,10.97430,11.9743,9.969210e+36,9.969210e+36,-0.012858,0.155495,1451696400
1,2016-01-01,10:00:00,-7.63623,279.864,-0.000810,2.41331,3.63232,0,0,1027.98,...,4.85847,9.969210e+36,9.969210e+36,11.00380,11.5140,9.969210e+36,9.969210e+36,-0.041467,0.154532,1451700000
2,2016-01-01,11:00:00,-7.21616,280.320,-0.000737,2.68491,3.57354,0,0,1028.01,...,4.41733,9.969210e+36,9.969210e+36,10.23570,11.2785,9.969210e+36,9.969210e+36,-0.005499,0.126797,1451703600
3,2016-01-01,12:00:00,-6.85010,280.740,-0.000613,2.92674,3.47312,0,0,1027.13,...,3.92283,9.969210e+36,9.969210e+36,9.58452,11.6122,9.969210e+36,9.969210e+36,0.013336,0.065372,1451707200
4,2016-01-01,13:00:00,-6.09811,281.544,-0.000488,2.97867,3.68480,0,0,1026.92,...,3.41606,9.969210e+36,9.969210e+36,9.47761,11.9958,9.969210e+36,9.969210e+36,0.039681,-0.016996,1451710800


In [57]:
datetimes=[datetime.datetime.fromtimestamp(item,pytz.utc) for item in round_trip_df['timestamp']]

In [58]:
datetimes[:5]

[datetime.datetime(2016, 1, 2, 1, 0, tzinfo=<UTC>),
 datetime.datetime(2016, 1, 2, 2, 0, tzinfo=<UTC>),
 datetime.datetime(2016, 1, 2, 3, 0, tzinfo=<UTC>),
 datetime.datetime(2016, 1, 2, 4, 0, tzinfo=<UTC>),
 datetime.datetime(2016, 1, 2, 5, 0, tzinfo=<UTC>)]